In [3]:
from typing import Annotated

from langchain.chat_models import init_chat_model
from langchain_tavily import TavilySearch
from langchain_core.messages import BaseMessage
from typing_extensions import TypedDict

from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition

import getpass
import os

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")
# sk-1feb7ffcf87f43a2a3feb12cc8349348
_set_env("DEEPSEEK_API_KEY")

# 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
_set_env("TAVILY_API_KEY")

class State(TypedDict):
    messages: Annotated[list, add_messages]


graph_builder = StateGraph(State)


tool = TavilySearch(max_results=2)
tools = [tool]
llm = init_chat_model("deepseek-chat")
llm_with_tools = llm.bind_tools(tools)


def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}


graph_builder.add_node("chatbot", chatbot)

tool_node = ToolNode(tools=[tool])
graph_builder.add_node("tools", tool_node)

graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,
)
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge(START, "chatbot")

memory = MemorySaver()
graph = graph_builder.compile(checkpointer=memory)

In [4]:
config = {"configurable": {"thread_id": "1"}}
events = graph.stream(
    {
        "messages": [
            {
                "role": "user",
                "content": (
                    "I'm learning LangGraph. "
                    "Could you do some research on it for me?"
                ),
            },
        ],
    },
    config,
    stream_mode="values",
)
for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()

================================ Human Message =================================

I'm learning LangGraph. Could you do some research on it for me?
================================== Ai Message ==================================
Tool Calls:
  tavily_search (call_0_4a4dab67-3874-48b7-99f2-f31c740b74cf)
 Call ID: call_0_4a4dab67-3874-48b7-99f2-f31c740b74cf
  Args:
    query: LangGraph
    search_depth: advanced
================================= Tool Message =================================
Name: tavily_search

{"query": "LangGraph", "follow_up_questions": null, "answer": null, "images": [], "results": [{"url": "https://langchain-ai.github.io/langgraph/tutorials/introduction/", "title": "LangGraph Quickstart - GitHub Pages", "content": "[](https://langchain-ai.github.io/langgraph/tutorials/introduction/#__codelineno-9-1)Assistant: LangGraph is a library designed to help build stateful multi-agent applications using language models. It provides tools for creating workflows and state machin

In [5]:
events = graph.stream(
    {
        "messages": [
            {
                "role": "user",
                "content": (
                    "Ya that's helpful. Maybe I'll "
                    "build an autonomous agent with it!"
                ),
            },
        ],
    },
    config,
    stream_mode="values",
)
for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()

================================ Human Message =================================

Ya that's helpful. Maybe I'll build an autonomous agent with it!
================================== Ai Message ==================================

That sounds like a great project! LangGraph is well-suited for building autonomous agents, especially those involving multi-agent workflows or stateful interactions. Here’s how you can get started and some ideas for your project:

---

### **Steps to Build an Autonomous Agent with LangGraph**
1. **Install LangGraph**:
   ```bash
   pip install -U langgraph
   ```

2. **Understand the Core Concepts**:
   - **State Machines**: LangGraph uses graph-based workflows to model agent behavior. You define nodes (agents or tasks) and edges (transitions between nodes).
   - **Multi-Agent Coordination**: You can design agents to collaborate or compete, passing messages or states between them.
   - **Memory**: LangGraph supports long-term memory, which is useful for agents 

In [6]:
to_replay = None
for state in graph.get_state_history(config):
    print("Num Messages: ", len(state.values["messages"]), "Next: ", state.next)
    print("-" * 80)
    if len(state.values["messages"]) == 6:
        # We are somewhat arbitrarily selecting a specific state based on the number of chat messages in the state.
        to_replay = state

Num Messages:  6 Next:  ()
--------------------------------------------------------------------------------
Num Messages:  5 Next:  ('chatbot',)
--------------------------------------------------------------------------------
Num Messages:  4 Next:  ('__start__',)
--------------------------------------------------------------------------------
Num Messages:  4 Next:  ()
--------------------------------------------------------------------------------
Num Messages:  3 Next:  ('chatbot',)
--------------------------------------------------------------------------------
Num Messages:  2 Next:  ('tools',)
--------------------------------------------------------------------------------
Num Messages:  1 Next:  ('chatbot',)
--------------------------------------------------------------------------------
Num Messages:  0 Next:  ('__start__',)
--------------------------------------------------------------------------------


In [7]:
print(to_replay.next)
print(to_replay.config)

()
{'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f02c807-6fa9-61de-8006-45bb4ed20cbd'}}


直接使用to_replay.config即可加载对话，注意是重新运行replay这个节点，而不是replay.next（毕竟next是null了）

In [8]:
# The `checkpoint_id` in the `to_replay.config` corresponds to a state we've persisted to our checkpointer.
for event in graph.stream(None, to_replay.config, stream_mode="values"):
    if "messages" in event:
        event["messages"][-1].pretty_print()

================================== Ai Message ==================================

That sounds like a great project! LangGraph is well-suited for building autonomous agents, especially those involving multi-agent workflows or stateful interactions. Here’s how you can get started and some ideas for your project:

---

### **Steps to Build an Autonomous Agent with LangGraph**
1. **Install LangGraph**:
   ```bash
   pip install -U langgraph
   ```

2. **Understand the Core Concepts**:
   - **State Machines**: LangGraph uses graph-based workflows to model agent behavior. You define nodes (agents or tasks) and edges (transitions between nodes).
   - **Multi-Agent Coordination**: You can design agents to collaborate or compete, passing messages or states between them.
   - **Memory**: LangGraph supports long-term memory, which is useful for agents that need to remember past interactions.

3. **Define Your Agent's Workflow**:
   - Break down the agent's tasks into smaller steps (nodes).
   - U